Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
df2[:height] = convert(Vector{Float64}, df2[:height]);
df2[:weight] = convert(Vector{Float64}, df2[:weight]);
df2[:weight_s] = (df2[:weight] .- mean(df2[:weight])) / std(df2[:weight]);
df2[:weight_s2] = df2[:weight_s] .^ 2;

Define the Stan language model

In [5]:
weightsmodel = "
data{
    int N;
    real height[N];
    real weight_s2[N];
    real weight_s[N];
}
parameters{
    real a;
    real b1;
    real b2;
    real sigma;
}
model{
    vector[N] mu;
    sigma ~ uniform( 0 , 50 );
    b2 ~ normal( 0 , 10 );
    b1 ~ normal( 0 , 10 );
    a ~ normal( 178 , 100 );
    for ( i in 1:N ) {
        mu[i] = a + b1 * weight_s[i] + b2 * weight_s2[i];
    }
    height ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["a", "b1", "b2", "sigma"],
model=weightsmodel,  output_format=:mcmcchain);

=====> /Users/rob/.julia/dev/StatisticalRethinking/scripts/04


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.stan will be updated.



Input data for cmdstan

In [7]:
heightsdata = Dict("N" => size(df2, 1), "height" => df2[:height],
"weight_s" => df2[:weight_s], "weight_s2" => df2[:weight_s2]);

Sample using cmdstan

In [8]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
CmdStanDir=CMDSTAN_HOME);



--- Translating Stan model to C++ code ---
bin/stanc  --o=/Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.hpp /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.stan
Model name=weights_model
Input file=/Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.stan
Output file=/Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.hpp
clang++ -std=c++1y -Wno-unknown-warning-option -Wno-tautological-compare -Wno-sign-compare      -O3 -I src -I stan/src -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.3 -I stan/lib/stan_math/lib/boost_1.66.0 -I stan/lib/stan_math/lib/sundials_3.1.0/include    -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -DBOOST_PHOENIX_NO_VARIADIC_EXPRESSION     -c -MT /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.o -MT /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights -include /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights

Describe the draws

In [9]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
           Mean         SD       Naive SE       MCSE      ESS
    a 154.615010000 0.35244157 0.0055725906 0.0067249910 1000
   b1   5.843281385 0.27239073 0.0043068755 0.0039288460 1000
   b2  -0.015285832 0.22371414 0.0035372312 0.0039201098 1000
sigma   5.111063883 0.19432321 0.0030725197 0.0025384738 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%       97.5%   
    a 153.91697500 154.383000 154.61300000 154.8460000 155.3200250
   b1   5.32747100   5.656025   5.83969500   6.0228025   6.3848375
   b2  -0.45379897  -0.163660  -0.01242305   0.1334930   0.4212011
sigma   4.75516400   4.978700   5.10421500   5.2395975   5.4989153



End of `m4.5s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*